In [3]:
# default_exp config
# export
from pathlib import Path
from shutil import which
import yaml
import os
import time
import getpass
import secrets

In [11]:
# export
import hashlib, binascii, os

def _hash_password(password):
    """Hash a password for storing."""
    salt = hashlib.sha256(os.urandom(60)).hexdigest().encode('ascii')
    pwdhash = hashlib.pbkdf2_hmac('sha512', password.encode('utf-8'), 
                                salt, 100000)
    pwdhash = binascii.hexlify(pwdhash)
    return (salt + pwdhash).decode('ascii')

def _verify_password(stored_password, provided_password):
    """Verify a stored password against one provided by user"""
    salt = stored_password[:64]
    stored_password = stored_password[64:]
    pwdhash = hashlib.pbkdf2_hmac('sha512', 
                                  provided_password.encode('utf-8'), 
                                  salt.encode('ascii'), 
                                  100000)
    pwdhash = binascii.hexlify(pwdhash).decode('ascii')
    return pwdhash == stored_password

In [15]:
# export

def is_tool(name):
    """Check whether `name` is on PATH and marked as executable."""
    return which(name) is not None

def guess_editor():
    editors = ['notepad','nano','vi']
    return next(e for e in editors if (is_tool(e)))

class Config():
    config_file = Path(os.getenv('QS_CONFIG', f'~/.config/qs.yaml')).expanduser() 

    def __init__(self):
        self.config_file.parent.mkdir(parents=True, exist_ok=True)
        if not self.config_file.exists(): self.create_config()
        self.d = self.load_config()
           
    def __getattr__(self, k):
        if k=='d': raise AttributeError
        return self.d[k]
                       
    def create_config(self):
        config = {'version': 1.0, 
                  'projects': {},
                  'editor': guess_editor(),
                  'secret': secrets.token_hex(32),  
                  'passwords': {}}
        
        self.save_config(config)
                
    def load_config(self):
        with self.config_file.open() as f:
            return yaml.load(f, Loader=yaml.FullLoader)
                    
    def save_config(self, config):
        self.config_file.parent.mkdir(parents=True, exist_ok=True)
        with self.config_file.open('w') as f:
            yaml.dump(config, f)

    def set_password(self, user, password):
        self.d[user] = _hash_password(password)
        self.save_config(self.d)
    
    def verify_password(self, user, password):
        return _verify_password(self.d.get(user, ''), password)

config = Config()

In [6]:
c = Config()
c.d

{'editor': 'nano',
 'passwords': {},
 'projects': {},
 'secret': '6c68ec4578bff79990398d703e1e1f5db7b852ae7cdb7e7cac1b25861f21f0c0',
 'version': 1.0}

In [7]:
c.create_config()

In [8]:
is_tool('vi')

True